# 🚀 NVIDIA Nemotron 3 Nano  - Simple Usage Guide

This notebook provides a streamlined guide to using **NVIDIA Nemotron 3 Nano**, including:

1. **Basic Usage** - Simple API calls with the OpenAI-compatible endpoint
2. **Reasoning Modes** - Toggle thinking ON/OFF
3. **LangChain v1.0 Agent** - Build a research assistant with web search and conversation memory
4. **Multi-Agent System** - Coordinate multiple specialist agents (Search, Report Writer, Quality Reviewer)

## Prerequisites

- An OpenRouter key ([Get one here](https://openrouter.ai/settings/keys))
- Optional: Tavily API key for web search ([Get one here](https://tavily.com/))


## 📦 Installation

Install all required dependencies:


## 🔑 API Key Configuration

Enter your OpenRouter API key when prompted. This key is used for both the OpenAI-compatible API and LangChain integration.

In [1]:
import os
import getpass

# Set OpenRouter API key (used by both OpenAI client and LangChain)
if not os.environ.get("OPENROUTER_API_KEY"):
    os.environ["OPENROUTER_API_KEY"] = getpass.getpass("Enter your OpenRouter API Key: ")

print("✅ OpenRouter API key configured!")


✅ OpenRouter API key configured!


---

# Part 1: Basic Usage with OpenAI Client

NVIDIA's API is OpenAI-compatible, making it easy to use with the standard OpenAI Python SDK.


In [2]:
from openai import OpenAI

# Create client pointing to OpenRouter's API
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ["OPENROUTER_API_KEY"]
)

# Model identifier (OpenRouter format)
MODEL = "nvidia/nemotron-3-nano-30b-a3b"

print(f"✅ Client configured for model: {MODEL}")


✅ Client configured for model: nvidia/nemotron-3-nano-30b-a3b


### Simple Request

Let's start with a basic request to the model:


In [4]:
# Simple non-streaming request
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant. Be concise."},
        {"role": "user", "content": "What is NVIDIA? Answer in 2-3 sentences."}
    ],
    temperature=1,
    top_p=1,
    max_tokens=512
)

print(response.choices[0].message.content)



NVIDIA is a technology company best known for designing graphics processing units (GPUs) that accelerate computer graphics, gaming, and data‑intensive tasks. In recent years, it has expanded into AI, cloud computing, automotive, and edge computing, offering hardware and software platforms that power everything from supercomputing to deep‑learning inference.


---

## 🧠 Reasoning Modes

Nemotron Nano V3 supports two reasoning modes:

| Mode | Description | Best For |
|------|-------------|----------|
| **Reasoning ON** | Outputs chain-of-thought wrapped in thinking tokens | Complex problems, math, logic |
| **Reasoning OFF** | Jumps directly to the final answer | Simple queries, faster responses |

Use `enable_thinking` in `extra_body` to toggle modes.


### Reasoning ON (with thinking)


In [5]:
# Helper function to display streaming response with reasoning
def stream_with_reasoning(completion):
    """Stream and display response, separating reasoning from final answer."""
    reasoning = ""
    answer = ""
    
    for chunk in completion:
        delta = chunk.choices[0].delta
        
        # Collect reasoning content (gray text)
        if hasattr(delta, 'reasoning_content') and delta.reasoning_content:
            reasoning += delta.reasoning_content
            print(f"\033[90m{delta.reasoning_content}\033[0m", end="", flush=True)
        
        # Collect answer content (normal text)
        if delta.content:
            answer += delta.content
            print(delta.content, end="", flush=True)
    
    print()  # New line at end
    return reasoning, answer


In [6]:
# Reasoning ON - The model thinks through the problem
# Best with temperature=1.0, top_p=1.0 for optimal performance

completion = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful math assistant."},
        {"role": "user", "content": "What is 15% of 240?"}
    ],
    extra_body={
        "chat_template_kwargs": {
            "enable_thinking": True,
        }
    },
    temperature=1.0,
    top_p=1.0,
    max_tokens=2048,
    stream=True
)

print("🧠 Reasoning ON (gray = thinking, white = answer):\n")
reasoning, answer = stream_with_reasoning(completion)


🧠 Reasoning ON (gray = thinking, white = answer):

User asks a simple math question: "What is 15% of 240?" Compute 0.15*240 = 36. Answer: 36.

15 % of 240 is  

\[
0.15 \times 240 = 36.
\] 

So the result is **36**.


### Reasoning OFF (direct answer)


In [7]:
# Reasoning OFF - Direct answer without thinking
# Best with temperature=0 for deterministic output

completion = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful math assistant."},
        {"role": "user", "content": "What is 15% of 240?"}
    ],
    extra_body={
        "chat_template_kwargs": {
            "enable_thinking": False,
        }
    },
    temperature=0,
    max_tokens=500,
    stream=True
)

print("⚡ Reasoning OFF (direct answer):\n")
_, answer = stream_with_reasoning(completion)


⚡ Reasoning OFF (direct answer):

To find **15% of 240**, follow these steps:

1. **Convert the percentage to a decimal**:  
   Divide 15 by 100 → **0.15**.

2. **Multiply the decimal by the number**:  
   **0.15 × 240 = 36**.

**Answer**: 15% of 240 is **36**.  

*Quick check*:  
- 10% of 240 = 24  
- 5% of 240 = 12  
- 15% = 10% + 5% → 24 + 12 = **36**. ✅


---

# Part 2: LangChain Research Agent 🔍

Now let's build something more powerful - a **research assistant** that can:
- 🌐 Search the web for current information
- 💬 Remember conversation history (memory)
- 🤖 Use NVIDIA Nemotron as its brain

We'll use LangChain's agent framework with the NVIDIA integration.


## Setup LangChain with OpenRouter


In [ ]:
from langchain_openai import ChatOpenAI

# Initialize the model via LangChain using OpenRouter endpoint
# Using ChatOpenAI with OpenRouter's base_url for OpenAI-compatible access
llm = ChatOpenAI(
    model="nvidia/nemotron-3-nano-30b-a3b",
    api_key=os.environ["OPENROUTER_API_KEY"],
    base_url="https://openrouter.ai/api/v1",
    temperature=1,
    top_p=1,
    max_tokens=4096,
)

# Test the connection
test_response = llm.invoke("Say 'Hello from NVIDIA!' in exactly those words.")
print(f"✅ LangChain + OpenRouter connected!\n{test_response.content}")

/home/chris/Code/NVIDIA/Nemotron 3 Nano - Simple Usage Guide/.venv/lib/python3.13/site-packages/langchain_nvidia_ai_endpoints/_common.py:241: UserWarning: Model private/nvidia/nemotron-nano-3-30b-a3b is unknown, check `available_models`. Inference may fail.
  warnings.warn(


✅ LangChain v1.0 + NVIDIA connected!

Hello from NVIDIA!


## 🌐 Web Search Tool

We'll use **DuckDuckGo** for web search (free, no API key required). 


In [ ]:
from langchain.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

# Create a DuckDuckGo search instance
_ddg_search = DuckDuckGoSearchRun()

# Wrap it with the @tool decorator for LangChain v1.0
@tool
def web_search(query: str) -> str:
    """Search the web for current information on any topic.
    
    Use this tool when you need up-to-date information about
    recent events, news, technology, or any topic that may
    have changed since your training data.
    
    Args:
        query: The search query to look up on the web
    """
    return _ddg_search.invoke(query)

# Test the search tool
test_search = web_search.invoke("What is Nemotron?")
print("🔍 Search test result (truncated):")
print(test_search[:500] + "..." if len(test_search) > 500 else test_search)

🔍 Search test result (truncated):
NVIDIA Nemotron ™ is a family of open models with open weights, training data, and recipes, delivering leading efficiency and accuracy for building specialized AI agents. So, what is Nemotron ? In short, Nemotron -4 340B is a decent-sized Large Language Model (LLM) that excels at crucial tasks in today’s world. A Predictable Architecture. What is Nemotron ? NVIDIA Nemotron ™ is a family of open, high-efficiency models with fully transparent training data, weights, and recipes. What Is NeMotron 3...


## 🤖 Create the Research Agent with Memory

Now we'll create an agent that:
1. Uses NVIDIA Nemotron as its LLM
2. Can search the web for information
3. Remembers conversation history within a session


In [15]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

# System prompt for our research assistant (LangChain v1.0 style)
SYSTEM_PROMPT = """You are a helpful research assistant powered by NVIDIA Nemotron.

Your capabilities:
- Search the web for current, up-to-date information
- Provide well-researched, accurate answers
- Remember our conversation context

Guidelines:
- Always search the web when asked about recent events, news, or current information
- Cite your sources when providing factual information
- Be concise but thorough
- If you're unsure, say so and search for more information"""

# Create memory checkpointer (LangChain v1.0 uses LangGraph's InMemorySaver)
checkpointer = InMemorySaver()

# Create the agent using LangChain v1.0's create_agent
# This is the new, simplified API that replaces create_tool_calling_agent + AgentExecutor
agent = create_agent(
    model=llm,                      # The NVIDIA model
    tools=[web_search],             # List of tools (using @tool decorated functions)
    system_prompt=SYSTEM_PROMPT,    # System instructions
    checkpointer=checkpointer,      # Memory/persistence
)

print("✅ LangChain v1.0 Research Agent with memory created!")


✅ LangChain v1.0 Research Agent with memory created!


## 🔬 Research Session Demo

Let's test our agent with a multi-turn research conversation. The agent will remember what we've discussed!


In [16]:
# Helper function for clean research queries (LangChain v1.0 style)
def research(query: str, thread_id: str = "default") -> str:
    """Send a query to the research agent and return the response.
    
    In LangChain v1.0, we use thread_id for conversation persistence
    instead of session_id. The agent automatically manages message history.
    """
    # LangChain v1.0 uses this config format for memory
    config = {"configurable": {"thread_id": thread_id}}
    
    # Invoke with messages format
    result = agent.invoke(
        {"messages": [{"role": "user", "content": query}]},
        config=config
    )
    
    # Extract the final response from messages
    # The last message contains the assistant's response
    messages = result.get("messages", [])
    if messages:
        return messages[-1].content
    return "No response generated."

def print_response(query: str, response: str):
    """Pretty print a research query and response."""
    print(f"\n{'='*60}")
    print(f"📝 Query: {query}")
    print(f"{'='*60}")
    print(f"\n{response}\n")


In [18]:
# Research Query 1: Initial question
# In LangChain v1.0, we use thread_id to maintain conversation state
THREAD_ID = "nvidia-research"

query1 = "What is Nemotron Nano 9B V2?"
response1 = research(query1, THREAD_ID)
print_response(query1, response1)


/home/chris/Code/NVIDIA/Nemotron 3 Nano - Simple Usage Guide/.venv/lib/python3.13/site-packages/langchain_nvidia_ai_endpoints/chat_models.py:842: UserWarning: Model 'private/nvidia/nemotron-nano-3-30b-a3b' is not known to support tools. Your tool binding may fail at inference time.
  warnings.warn(
/home/chris/Code/NVIDIA/Nemotron 3 Nano - Simple Usage Guide/.venv/lib/python3.13/site-packages/langchain_nvidia_ai_endpoints/chat_models.py:842: UserWarning: Model 'private/nvidia/nemotron-nano-3-30b-a3b' is not known to support tools. Your tool binding may fail at inference time.
  warnings.warn(



📝 Query: What is Nemotron Nano 9B V2?


**Nemotron‑Nano‑9B‑V2** is the second‑generation 9‑billion‑parameter language model in NVIDIA’s Nemotron‑Nano family. It is an open‑weight, general‑purpose LLM designed for both **reasoning** and **non‑reasoning** tasks, and it can be used in English as well as in several non‑English languages (German, French, Italian, Spanish, Japanese).

### Key characteristics
| Aspect | Details |
|--------|---------|
| **Model size** | 9 billion parameters |
| **Purpose** | Unified model for reasoning‑oriented chat and standard language tasks |
| **Training** | Trained **from scratch** by NVIDIA Research; the complete training pipeline (pre‑training data, recipes, checkpoint) is publicly released |
| **Reasoning style** | Generates an internal “reasoning trace” before producing the final answer, improving accuracy on complex queries |
| **Deployment** | Available as an open‑weight checkpoint that can run on a single GPU; compatible with NVIDIA NIM™ microserv

### View Conversation History

The agent stores the full conversation in memory. Let's peek at what's stored:


In [ ]:
config = {"configurable": {"thread_id": THREAD_ID}}
state = agent.get_state(config)

print("Conversation History (LangChain v1.0):")
print("-" * 40)

if state and state.values:
    messages = state.values.get("messages", [])
    for i, msg in enumerate(messages):
        role = msg.type.capitalize() if hasattr(msg, 'type') else "Unknown"
        content = msg.content if hasattr(msg, 'content') else str(msg)
        content = content[:200] + "..." if len(content) > 200 else content
        print(f"\n[{i+1}] {role}:")
        print(f"    {content}")
else:
    print("No conversation history found.")


Conversation History (LangChain v1.0):
----------------------------------------

[1] Human:
    What is Nemotron?

[2] Ai:
    

[3] Tool:
    The Nemotron model family, available as optimized NVIDIA NIM™ microservices , offers peak inference performance and flexible deployment options, ensuring superior security, privacy, and portability. 4...

[4] Ai:
    
**Nemotron** is NVIDIA’s family of open‑weight language models that are purpose‑built for high‑performance, flexible deployment — from data‑center GPUs to edge devices.  

- **Core idea:** A suite of...

[5] Human:
    What is Nemotron Nano 9B V2?

[6] Ai:
    

[7] Tool:
    NVIDIA- Nemotron - Nano - 9 B - v 2 is a general purpose reasoning and chat model intended to be used in English and coding languages. Other non-English languages (German, French, Italian, Spanish and...

[8] Ai:
    
**Nemotron‑Nano‑9B‑V2** is the second‑generation 9‑billion‑parameter language model in NVIDIA’s Nemotron‑Nano family. It is an open‑weight, gene

---

## Try Your Own Research!

Use the cell below to try your own research queries. Change the `SESSION_ID` to start a fresh conversation.


In [ ]:
# Try your own research query!
# Change the thread_id to start a fresh conversation

my_query = "What are the latest breakthroughs in renewable energy?"
my_thread = "my-research"  # Use a different thread_id for a new conversation

response = research(my_query, my_thread)
print_response(my_query, response)


---

# Part 3: Multi-Agent Research System 🤝

Now let's build a more sophisticated **multi-agent system** where specialized agents collaborate to produce high-quality research reports.

Based on [LangChain's multi-agent patterns](https://docs.langchain.com/oss/python/langchain/multi-agent), we'll implement the **Tool Calling (Supervisor) pattern** where:

| Agent | Role |
|-------|------|
| **Coordinator** | Orchestrates the workflow, decides which specialist to call |
| **Search Agent** | Specializes in finding relevant information from the web |
| **Report Writer** | Transforms research into well-structured reports |
| **Quality Reviewer** | Verifies helpfulness and suggests improvements |

```
┌─────────────────┐
│   Coordinator   │ ◄── User Query
│   (Supervisor)  │
└────────┬────────┘
         │
    ┌────┴────┬──────────┐
    ▼         ▼          ▼
┌───────┐ ┌────────┐ ┌──────────┐
│Search │ │Report  │ │Quality   │
│Agent  │ │Writer  │ │Reviewer  │
└───────┘ └────────┘ └──────────┘
```


## Step 1: Create Specialized Sub-Agents

Each sub-agent is a focused specialist with its own system prompt and capabilities.


In [21]:
# Create specialized sub-agents using LangChain v1.0's create_agent
# Each agent has a focused role and specialized prompt

# 1. SEARCH AGENT - Expert at finding information
search_agent = create_agent(
    model=llm,
    tools=[web_search],  # Has access to web search
    system_prompt="""You are a Search Specialist Agent.

Your ONLY job is to search the web and gather relevant information.

When given a research topic:
1. Formulate effective search queries
2. Search for multiple aspects of the topic
3. Compile the key findings
4. Return a structured summary of what you found

Always include the sources of your information.
Be thorough but focused - gather facts, not opinions."""
)

# 2. REPORT WRITER AGENT - Expert at writing reports
report_writer_agent = create_agent(
    model=llm,
    tools=[],  # No tools - pure writing capability
    system_prompt="""You are a Report Writer Specialist Agent.

Your ONLY job is to transform raw research into well-written reports.

When given research findings:
1. Organize information logically
2. Write clear, engaging prose
3. Include relevant sections (Overview, Key Points, Details, Conclusion)
4. Use markdown formatting for structure
5. Cite sources where appropriate

Focus on clarity and readability. Make complex topics accessible."""
)

# 3. QUALITY REVIEWER AGENT - Expert at evaluating helpfulness
quality_reviewer_agent = create_agent(
    model=llm,
    tools=[],  # No tools - pure evaluation capability
    system_prompt="""You are a Quality Reviewer Specialist Agent.

Your ONLY job is to evaluate if a report is helpful and complete.

When reviewing a report:
1. Check if it answers the original question
2. Verify key information is included
3. Assess clarity and readability
4. Identify any gaps or missing information
5. Provide a helpfulness score (1-10) with justification

Be constructive - if improvements are needed, explain what's missing.
If the report is good, confirm it's ready for the user."""
)

print("✅ Three specialized sub-agents created:")
print("   - Search Agent (with web_search tool)")
print("   - Report Writer Agent (writing specialist)")
print("   - Quality Reviewer Agent (evaluation specialist)")


✅ Three specialized sub-agents created:
   - Search Agent (with web_search tool)
   - Report Writer Agent (writing specialist)
   - Quality Reviewer Agent (evaluation specialist)


## Step 2: Wrap Sub-Agents as Tools

Following [LangChain's multi-agent pattern](https://docs.langchain.com/oss/python/langchain/multi-agent), we wrap each sub-agent as a tool that the coordinator can invoke.


In [22]:
# Wrap each sub-agent as a tool for the coordinator
# This is the "Tool Calling" multi-agent pattern from LangChain v1.0

@tool("search_specialist")
def call_search_agent(research_topic: str) -> str:
    """Call the Search Specialist to gather information from the web.
    
    Use this when you need to find current information, facts, or data
    about a topic. The search specialist will search the web and return
    a summary of findings.
    
    Args:
        research_topic: The topic or question to research
    """
    result = search_agent.invoke({
        "messages": [{"role": "user", "content": f"Research this topic: {research_topic}"}]
    })
    return result["messages"][-1].content


@tool("report_writer")
def call_report_writer(research_findings: str) -> str:
    """Call the Report Writer to create a well-structured report.
    
    Use this after gathering research to transform raw findings into
    a polished, readable report with proper structure and formatting.
    
    Args:
        research_findings: The raw research data and findings to write about
    """
    result = report_writer_agent.invoke({
        "messages": [{"role": "user", "content": f"Write a report based on these findings:\n\n{research_findings}"}]
    })
    return result["messages"][-1].content


@tool("quality_reviewer")
def call_quality_reviewer(report_to_review: str) -> str:
    """Call the Quality Reviewer to evaluate a report's helpfulness.
    
    Use this to verify that a report is complete, helpful, and ready
    for the user. The reviewer will score it and suggest improvements.
    
    Args:
        report_to_review: The report content to evaluate
    """
    result = quality_reviewer_agent.invoke({
        "messages": [{"role": "user", "content": f"Review this report for helpfulness and completeness:\n\n{report_to_review}"}]
    })
    return result["messages"][-1].content


print("✅ Sub-agents wrapped as tools:")
print("   - call_search_agent: For gathering information")
print("   - call_report_writer: For creating reports")
print("   - call_quality_reviewer: For evaluating quality")


✅ Sub-agents wrapped as tools:
   - call_search_agent: For gathering information
   - call_report_writer: For creating reports
   - call_quality_reviewer: For evaluating quality


## Step 3: Create the Coordinator (Supervisor) Agent

The coordinator orchestrates the workflow by deciding which specialist to call and when.


In [23]:
# Create the Coordinator (Supervisor) Agent
# This agent orchestrates the multi-agent workflow

COORDINATOR_PROMPT = """You are a Research Coordinator managing a team of specialist agents.

Your team consists of:
1. **Search Specialist** - Finds information from the web
2. **Report Writer** - Creates well-structured reports
3. **Quality Reviewer** - Evaluates helpfulness and completeness

WORKFLOW for research requests:
1. First, call the Search Specialist to gather information
2. Then, call the Report Writer to create a report from the findings
3. Finally, call the Quality Reviewer to verify the report is helpful
4. If the reviewer suggests improvements, you may iterate
5. Present the final report to the user

Always follow this workflow for research questions. Coordinate the specialists
efficiently and ensure the final output is high-quality and helpful.

Remember: You are the coordinator. Delegate tasks to specialists - don't try
to search or write reports yourself."""

# Create coordinator with access to all specialist tools
multi_agent_checkpointer = InMemorySaver()

coordinator = create_agent(
    model=llm,
    tools=[call_search_agent, call_report_writer, call_quality_reviewer],
    system_prompt=COORDINATOR_PROMPT,
    checkpointer=multi_agent_checkpointer,
)

print("✅ Multi-Agent Research System ready!")
print("\nWorkflow: User → Coordinator → Search → Writer → Reviewer → User")


✅ Multi-Agent Research System ready!

Workflow: User → Coordinator → Search → Writer → Reviewer → User


## Step 4: Run the Multi-Agent System

Let's test the full multi-agent workflow with a research question.


In [24]:
# Helper function for multi-agent research
def multi_agent_research(topic: str, thread_id: str = "multi-agent-default") -> str:
    """Run a research query through the multi-agent system.
    
    The coordinator will orchestrate:
    1. Search Agent → gather information
    2. Report Writer → create report
    3. Quality Reviewer → verify helpfulness
    """
    config = {"configurable": {"thread_id": thread_id}}
    
    print(f"🚀 Starting multi-agent research on: {topic}")
    print("-" * 50)
    
    result = coordinator.invoke(
        {"messages": [{"role": "user", "content": topic}]},
        config=config
    )
    
    # Get the final response
    final_response = result["messages"][-1].content
    return final_response

# Demo: Run a multi-agent research query
MULTI_AGENT_THREAD = "research-demo"

research_topic = "What are the key features and benefits of NVIDIA's Blackwell GPU architecture?"

print("=" * 60)
print("MULTI-AGENT RESEARCH DEMO")
print("=" * 60)

final_report = multi_agent_research(research_topic, MULTI_AGENT_THREAD)

print("\n" + "=" * 60)
print("FINAL REPORT")
print("=" * 60)
print(final_report)


MULTI-AGENT RESEARCH DEMO
🚀 Starting multi-agent research on: What are the key features and benefits of NVIDIA's Blackwell GPU architecture?
--------------------------------------------------


/home/chris/Code/NVIDIA/Nemotron 3 Nano - Simple Usage Guide/.venv/lib/python3.13/site-packages/langchain_nvidia_ai_endpoints/chat_models.py:842: UserWarning: Model 'private/nvidia/nemotron-nano-3-30b-a3b' is not known to support tools. Your tool binding may fail at inference time.
  warnings.warn(
/home/chris/Code/NVIDIA/Nemotron 3 Nano - Simple Usage Guide/.venv/lib/python3.13/site-packages/langchain_nvidia_ai_endpoints/chat_models.py:842: UserWarning: Model 'private/nvidia/nemotron-nano-3-30b-a3b' is not known to support tools. Your tool binding may fail at inference time.
  warnings.warn(
/home/chris/Code/NVIDIA/Nemotron 3 Nano - Simple Usage Guide/.venv/lib/python3.13/site-packages/langchain_nvidia_ai_endpoints/chat_models.py:842: UserWarning: Model 'private/nvidia/nemotron-nano-3-30b-a3b' is not known to support tools. Your tool binding may fail at inference time.
  warnings.warn(
/home/chris/Code/NVIDIA/Nemotron 3 Nano - Simple Usage Guide/.venv/lib/python3.13/site-packages/lang


FINAL REPORT

**NVIDIA Blackwell GPU Architecture – Key Features, Benefits, and Expanded Details**

---

## 1. Overview
Blackwell is NVIDIA’s newest **GPU micro‑architecture**, succeeding Hopper and Ada Lovelace. It is purpose‑built for two primary workloads:

| Workload | What Blackwell does |
|----------|----------------------|
| **AI training & inference** | Accelerates large‑scale model training (LLMs, diffusion, recommendation) and high‑throughput inference. |
| **High‑performance graphics** | Powers the upcoming **RTX 50‑series** graphics cards and the **RTX PRO 6000 Blackwell** workstation GPUs. |

---

## 2. Core Features (with technical specifics)

| Feature | Technical Detail | Why It Matters |
|---------|-----------------|----------------|
| **5th‑Generation Tensor Cores** | • Up to **1,000 TFLOPs** of FP8/FP16/TF32 compute per GPU.<br>• New matrix‑multiply instructions that fuse attention kernels. | Delivers dramatically higher AI‑compute throughput and efficiency. |
| **A

### View Multi-Agent Collaboration

Let's see how the agents collaborated - which tools were called and in what order:


In [25]:
# View the multi-agent collaboration history
config = {"configurable": {"thread_id": MULTI_AGENT_THREAD}}
state = coordinator.get_state(config)

print("Multi-Agent Collaboration Log:")
print("=" * 50)

if state and state.values:
    messages = state.values.get("messages", [])
    for i, msg in enumerate(messages):
        msg_type = msg.type.upper() if hasattr(msg, 'type') else "UNKNOWN"
        
        # Check if it's a tool call
        if msg_type == "AI" and hasattr(msg, 'tool_calls') and msg.tool_calls:
            for tc in msg.tool_calls:
                print(f"\n[{i+1}] 🔧 TOOL CALL: {tc.get('name', 'unknown')}")
                args = tc.get('args', {})
                arg_preview = str(args)[:100] + "..." if len(str(args)) > 100 else str(args)
                print(f"    Args: {arg_preview}")
        elif msg_type == "TOOL":
            tool_name = msg.name if hasattr(msg, 'name') else "unknown"
            content_preview = msg.content[:150] + "..." if len(msg.content) > 150 else msg.content
            print(f"\n[{i+1}] 📥 TOOL RESULT ({tool_name}):")
            print(f"    {content_preview}")
        elif msg_type == "HUMAN":
            print(f"\n[{i+1}] 👤 USER: {msg.content[:100]}...")
        elif msg_type == "AI":
            content_preview = msg.content[:150] + "..." if len(msg.content) > 150 else msg.content
            print(f"\n[{i+1}] 🤖 COORDINATOR: {content_preview}")
else:
    print("No collaboration history found.")


Multi-Agent Collaboration Log:

[1] 👤 USER: What are the key features and benefits of NVIDIA's Blackwell GPU architecture?...

[2] 🔧 TOOL CALL: search_specialist
    Args: {'research_topic': 'NVIDIA Blackwell GPU architecture key features and benefits'}

[3] 📥 TOOL RESULT (search_specialist):
    
**NVIDIA Blackwell GPU Architecture – Key Features & Benefits**  

---

### 1. Overview  
Blackwell is NVIDIA’s latest graphics‑processing‑unit micro...

[4] 🔧 TOOL CALL: quality_reviewer
    Args: {'report_to_review': 'NVIDIA Blackwell GPU Architecture – Key Features & Benefits\n\n---\n\n### 1. O...

[5] 📥 TOOL RESULT (quality_reviewer):
    
**Overall helpfulness score: 8 / 10**

---

### ✅ What the report does well  

| Criterion | Evaluation | Why it earns points |
|-----------|--------...

[6] 🤖 COORDINATOR: 
**NVIDIA Blackwell GPU Architecture – Key Features, Benefits, and Expanded Details**

---

## 1. Overview
Blackwell is NVIDIA’s newest **GPU micro‑ar...


## Try Your Own Multi-Agent Research

Run any research topic through the multi-agent system:


In [ ]:
# Try your own multi-agent research!
# Change the topic and thread_id as desired

my_topic = "What are the latest advancements in quantum computing in 2024?"
my_thread = "my-multi-agent-research"

report = multi_agent_research(my_topic, my_thread)
print("\n" + "=" * 60)
print("YOUR RESEARCH REPORT")
print("=" * 60)
print(report)


---

# Summary

In this notebook, we covered:

### Part 1: Basic Usage
- Setting up the OpenAI-compatible client via **OpenRouter**
- **Reasoning ON**: Chain-of-thought reasoning (best for complex problems)
- **Reasoning OFF**: Direct answers (best for simple queries)

### Part 2: LangChain v1.0 Single Agent
- Used the new **`create_agent`** API
- Integrated NVIDIA Nemotron via **OpenRouter** using `ChatOpenAI` with custom `base_url`
- Added **web search** capability with `@tool` decorator
- Implemented **conversation memory** using `InMemorySaver` checkpointer

### Part 3: Multi-Agent System
- Created **specialized sub-agents** (Search, Report Writer, Quality Reviewer)
- Wrapped sub-agents as **tools** following [LangChain's multi-agent pattern](https://docs.langchain.com/oss/python/langchain/multi-agent)
- Built a **Coordinator (Supervisor)** agent to orchestrate the workflow
- Demonstrated the full **research pipeline**: Search → Write → Review

## Multi-Agent Architecture

```
┌─────────────────────────────────────────────────────┐
│                    COORDINATOR                       │
│   (Decides which specialist to call and when)        │
└─────────────┬─────────────┬─────────────┬───────────┘
              │             │             │
     ┌────────▼────┐  ┌─────▼──────┐  ┌───▼────────┐
     │   SEARCH    │  │   REPORT   │  │  QUALITY   │
     │   AGENT     │  │   WRITER   │  │  REVIEWER  |
     │ (web_search)│  │ (writing)  │  │(evaluation)│
     └─────────────┘  └────────────┘  └────────────┘
```

## Key LangChain v1.0 Patterns Used

| Pattern | Description |
|---------|-------------|
| `create_agent` | New simplified agent creation API |
| `@tool` decorator | Define tools from functions |
| `InMemorySaver` | Thread-based conversation memory |
| **Tool Calling** | Sub-agents wrapped as tools for orchestration |

## Resources

- [OpenRouter Documentation](https://openrouter.ai/docs)
- [LangChain Chat Completions API (OpenRouter)](https://docs.langchain.com/oss/python/integrations/chat/index)
- [LangChain v1.0 Release Notes](https://docs.langchain.com/oss/python/releases/langchain-v1)
- [Multi-Agent Patterns](https://docs.langchain.com/oss/python/langchain/multi-agent)
- [Supervisor Agent Tutorial](https://docs.langchain.com/oss/python/langchain/supervisor)
- [LangGraph for Advanced Agents](https://langchain-ai.github.io/langgraph/)
